# [75.06 / 95.58] Organización de Datos
## Trabajo Práctico 1: Análisis Exploratorio de Datos
### Anexo: Creación de dataframes

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**https://github.com/FdelMazo/7506-Datos**

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/events.csv", low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

### brands.csv

In [ ]:
checked = ['iphone', 'samsung', 'motorola', 'lenovo', 'sony', 'lg', 'ipad', 'asus', 'quantum', 'blackberry']
model_parsed = df['model'].dropna().map(lambda x: x.lower())
model_parsed = model_parsed.map(lambda x: x.split())

def find_brand(model):
    for str in model:
        if str in checked:
            return str
    return "other"


df['brand'] = model_parsed.map(find_brand)
df['brand'] = df['brand'].astype('category')

In [ ]:
df[['model', 'brand']].dropna().drop_duplicates().to_csv('data/brands.csv', index=False)

### os.csv

In [ ]:
df['operating_system_version'].unique()

In [ ]:
checked = ['windows', 'android', 'linux', 'mac', 'ios', 'ubuntu', 'chrome os', 'tizen', 'other']
os_version_parsed = df['operating_system_version'].dropna().map(lambda x: x.lower())

def find_os(os_version):
    for os in checked:
        if os in os_version:
            return os
    return "another"


df['operating_system'] = os_version_parsed.map(find_os)
df['operating_system'] = df['operating_system'].astype('category')

In [ ]:
df[['operating_system_version', 'operating_system']].dropna().head(10)
df[df['operating_system'] == 'another'][['operating_system_version', 'operating_system']].head(10)

In [ ]:
df['operating_system'].value_counts()

In [ ]:
df[['operating_system_version', 'operating_system']].dropna().drop_duplicates().to_csv('data/os.csv', index=False)

In [ ]:
df[['operating_system_version', 'operating_system']].head(10)

### browsers.csv

In [ ]:
checked = ['mobile safari', 'chrome mobile', 'ie mobile', 'firefox mobile', 'edge mobile', 'opera mobile',
           'mobile', 'chrome', 'android', 'opera', 'ie', 'firefox', 
           'facebook', 'samsung', 'chromium', 'edge', 'yandex', 'uc', 
           'other', 'safari', 'puffin', 'maxthon', 'vivaldi']
browser_version_parsed = df['browser_version'].dropna().map(lambda x: x.lower())

def find_browser(browser_version):
    for browser in checked:
        if browser in browser_version:
            return browser
    return "other"


df['browser'] = browser_version_parsed.map(find_browser)
df['browser'] = df['browser'].astype('category')

In [ ]:
df[(df['browser'] == 'mobile')][['browser_version', 'browser']]

In [ ]:
df[['browser_version', 'browser']].dropna().head(10)
display(df['browser'].value_counts())
df[(df['browser'] == 'another')][['browser_version', 'browser']]

In [ ]:
df[['browser_version', 'browser']].dropna().drop_duplicates().to_csv('data/browsers.csv', index=False)

In [ ]:
df[['browser_version', 'browser']].head(20)

#### Colors

# Funnels

Definimos una sesión como una serie de eventos por usuario, los cuales están todos con menos de 30 minutos de inactividad entre el actual y el anterior.

In [ ]:
funnel = df.sort_values(['person', 'timestamp'])

In [ ]:
funnel['time_diff'] = funnel.groupby('person')['timestamp'].diff()
funnel['time_diff'] = funnel['time_diff'].fillna(0)
funnel['time_diff_min'] = funnel['time_diff'] / np.timedelta64(1, 'm')

In [ ]:
THRESHOLD = 30 # minutes

funnel['new_session'] = funnel['time_diff_min'] > THRESHOLD
funnel['session_id'] = funnel.groupby('person')['new_session'].cumsum()
funnel['session_id'] = funnel['session_id'].astype('int')

In [ ]:
gb = funnel.groupby(['person', 'session_id'])

funnel['session_cumno'] = gb.cumcount()
funnel['session_total_events'] = gb['session_cumno'].transform(lambda x: x.size)
funnel['session_first'] = funnel['session_cumno'] == 0
funnel['session_last'] = funnel['session_cumno'] == (-1+funnel['session_total_events'])

In [ ]:
cols = ['person', 'timestamp', 'time_diff_min', \
        'session_id', 'event', 'session_total_events', \
        'session_cumno', 'session_first', 'session_last']
funnel[cols]

In [ ]:
funnel['is_conversion'] = funnel['event'] == 'conversion'
gb = funnel.groupby(['person', 'session_id'])
funnel['session_conversion'] = gb['is_conversion'].transform(lambda x: x.sum())
funnel['session_conversion'] = funnel['session_conversion'] > 0

In [ ]:
funnel2[funnel2['session_conversion']][cols].head(10)

In [ ]:
funnel['is_checkout'] = funnel['event'] == 'checkout'
gb = funnel.groupby(['person', 'session_id'])
funnel['session_checkout'] = gb['is_checkout'].transform(lambda x: x.sum())
funnel['session_checkout'] = funnel['session_checkout'] > 0

In [ ]:
funnel2[funnel2['session_checkout']][cols].head(10)

In [ ]:
funnel['ad_origin'] = (funnel['event'] == 'ad campaign hit') & funnel['session_first']
gb = funnel.groupby(['person', 'session_id'])
funnel['session_organic'] = gb['add_origin'].transform(lambda x: x.sum())
funnel['session_organic'] = funnel['session_organic'] == 0

In [ ]:
funnel[cols]

In [ ]:
cols_csv = ['time_diff_min', \
        'session_id', 'session_total_events', \
        'session_cumno', 'session_first', 'session_last', \
        'session_conversion', 'session_checkout', 'session_organic']

funnel[cols_csv].to_csv('data/sessions.csv', index=False)

In [ ]:
df = funnel

#### Funnel graph

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

from __future__ import division

plotly.tools.set_credentials_file(username='datatouille', api_key='GJ7Foc8nFWf23VZfzFSK')

In [ ]:
dfunnel = df[\
    (df['event'] == 'generic listing') | \
    (df['event'] == 'brand listing') | \
    (df['event'] == 'searched products') | \
    (df['event'] == 'viewed product') | \
    (df['event'] == 'checkout') | \
    (df['event'] == 'conversion') \
]

# color for each segment
colors = ['rgb(63,92,128)', 'rgb(90,131,182)', 'rgb(255,255,255)', 'rgb(127,127,127)', 'rgb(84,73,75)']
colors = ['rgb(63,92,128)', 'rgb(84,73,75)']

In [ ]:
df.loc[df['skus'].dropna().index]['event'].value_counts()

In [ ]:
dfunnel = dfunnel.groupby(['event', 'session_organic']).size().unstack(level='session_organic')
dfunnel

In [ ]:
def group_lists(event):
    if event == 'brand listing' or event == 'generic listing' or event == 'searched products':
        return 'listed'
    return event

dfunnel.columns = ['Ad', 'Organic']
dfunnel = dfunnel.groupby(dfunnel.index.map(lambda x: group_lists(x))).sum()
dfunnel = dfunnel.reindex(['listed', \
                 'viewed product', 'checkout', 'conversion'])
dfunnel = np.log(dfunnel)

In [ ]:
dfunnel

In [ ]:
total = [sum(row[1]) for row in dfunnel.iterrows()]

In [ ]:
n_phase, n_seg = dfunnel.shape

In [ ]:
plot_width = 500
unit_width = plot_width / total[0]
 
phase_w = [int(value * unit_width) for value in total]
 
# height of a section and difference between sections 
section_h = 100
section_d = 10

# shapes of the plot
shapes = []
 
# plot traces data
data = []
 
# height of the phase labels
label_y = []

In [ ]:
height = section_h * n_phase + section_d * (n_phase-1)

# rows of the DataFrame
df_rows = list(dfunnel.iterrows())

# iteration over all the phases
for i in range(n_phase):
    # phase name
    row_name = dfunnel.index[i]
    
    # width of each segment (smaller rectangles) will be calculated
    # according to their contribution in the total users of phase
    seg_unit_width = phase_w[i] / total[i]
    seg_w = [int(df_rows[i][1][j] * seg_unit_width) for j in range(n_seg)]
    
    # starting point of segment (the rectangle shape) on the X-axis
    xl = -1 * (phase_w[i] / 2)
    
    # iteration over all the segments
    for j in range(n_seg):
        # name of the segment
        seg_name = dfunnel.columns[j]
        
        # corner points of a segment used in the SVG path
        points = [xl, height, xl + seg_w[j], height, xl + seg_w[j], height - section_h, xl, height - section_h]
        path = 'M {0} {1} L {2} {3} L {4} {5} L {6} {7} Z'.format(*points)
        
        shape = {
                'type': 'path',
                'path': path,
                'fillcolor': colors[j],
                'line': {
                    'width': 1,
                    'color': colors[j]
                }
        }
        shapes.append(shape)
        
        # to support hover on shapes
        hover_trace = go.Scatter(
            x=[xl + (seg_w[j] / 2)],
            y=[height - (section_h / 2)],
            mode='markers',
            marker=dict(
                size=min(seg_w[j]/2, (section_h / 2)),
                color='rgba(255,255,255,1)'
            ),
            text="Segment : %s" % (seg_name),
            name="Value : %d" % (dfunnel[seg_name][row_name])
        )
        data.append(hover_trace)
        
        xl = xl + seg_w[j]

    label_y.append(height - (section_h / 2))

    height = height - (section_h + section_d)

In [ ]:
# For phase names
label_trace = go.Scatter(
    x=[-350]*n_phase,
    y=label_y,
    mode='text',
    text=dfunnel.index.tolist(),
    textfont=dict(
        color='rgb(200,200,200)',
        size=11
    )
)

data.append(label_trace)
 
# For phase values (total)
value_trace = go.Scatter(
    x=[350]*n_phase,
    y=label_y,
    mode='text',
    text=total,
    textfont=dict(
        color='rgb(200,200,200)',
        size=11
    )
)

In [ ]:
layout = go.Layout(
    title="<b>Funnel (log) [Ad:Azul | Orgánico:Marrón]</b>",
    titlefont=dict(
        size=20,
        color='rgb(230,230,230)'
    ),
    hovermode='closest',
    shapes=shapes,
    showlegend=False,
    paper_bgcolor='rgba(44,58,71,1)',
    plot_bgcolor='rgba(44,58,71,1)',
    xaxis=dict(
        showticklabels=False,
        zeroline=False,
    ),
    yaxis=dict(
        showticklabels=False,
        zeroline=False
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)